# Multi Resolution Simulation

### Classes and modules

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

GPU Ocean-modules:

In [ ]:
from gpuocean.utils import IPythonMagic, NetCDFInitialization

In [ ]:
%cuda_context_handler gpu_ctx

## Selecting loc

As initial and boundary conditions to the simulation, we use data from the Norkyst800 s-level files:

In [ ]:
nk800_url  = "/sintef/data/NorKyst800/NorKyst-800m_ZDEPTHS_his.an.2019071600.nc"

nk800_data_args = NetCDFInitialization.removeMetadata(NetCDFInitialization.getInitialConditionsNorKystCases(nk800_url, "lofoten", download_data=False, norkyst_data=True))

In [ ]:
plt.imshow(nk800_data_args["H"])

In [ ]:
nf160_url = "/sintef/data/NorFjords160/A10/norfjords_160m_his.nc4_2019071601-2019071700"

nf160_data_args = NetCDFInitialization.removeMetadata(NetCDFInitialization.getInitialConditions(nf160_url,50, 1950, 50, 1100, download_data=False, norkyst_data=False))

In [ ]:
plt.imshow(nf160_data_args["H"])

In [ ]:
import netCDF4
from gpuocean.utils import GPUOcean_runDeterministic


try:
    ncfile = netCDF4.Dataset(nf160_url)
    lon_rho = ncfile.variables['lon_rho'][:]
    lat_rho = ncfile.variables['lat_rho'][:]
except Exception as e:
    raise e
finally:
    ncfile.close()
    
X_norkyst, Y_norkyst, proj = GPUOcean_runDeterministic.getXYproj(nk800_url)
    
X_norfjords, Y_norfjords = proj(lon_rho, lat_rho, inverse = False) #Norfjords within norkyst800(total domain)
X_norfjords = X_norfjords[0]
Y_norfjords = Y_norfjords[:,0] 

In [ ]:
X_norkyst

In [ ]:
X_norfjords

In [ ]:
import xarray as xr
nk800_ds = xr.open_dataset(nk800_url)
nk800_ds

In [ ]:
nf160_ds = xr.open_dataset(nf160_url)
nf160_ds

In [ ]:
# conda install -c conda-forge ncplot
import ncplot.xarray
nk800_ds.ncplot.view("zeta")

In [ ]:
nf160_ds.ncplot.view()